In [24]:
from __future__ import unicode_literals
from hazm import * 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import math

In [25]:
train = pd.read_csv('vehicles.csv')

train = pd.DataFrame(train)

normalizer = Normalizer()
lemmatizer = Lemmatizer()
stop_words = open('stopwords', 'r').read().splitlines()
stop_words = [lemmatizer.lemmatize(w) for w in stop_words]
stop_words.append('\n')
stop_words.append('\r')


for index,row in train.iterrows():
    normalizer.normalize(row['description'])
    for w in word_tokenize(row['description']):
        word = lemmatizer.lemmatize(w)
        if word not in hazm.utils.stopwords_list:
            print("kir")
    


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 227: character maps to <undefined>

,brand,category,created_at,description,image_count,mileage,price,title,year
0,NaN,heavy,Tuesday 07PM,سریال 43j$NUMبدون شکستگی سه حلقه لاستیک نو یک ...,4,NaN,-1,لودر کاتر پیلار 950,NaN
1,سایر,light,Wednesday 04AM,دوگانه ال پی جی بیمه با تخفیف دارد \nتودوزی نو...,3,180000.0,-1,پاترول شش سیلندر چهار درب,1366
2,پژو ۴۰۵::Peugeot 405,light,Wednesday 11AM,پژو 2000فنی تازه تعمیر بدنه بدون خطو خش تخیف ب...,0,290000.0,8500000,پژو مدل 81 مشکی,1381
3,نیسان::Nissan,light,Wednesday 01PM,سلام.\nماشین کاملا سالم همه چی فابریک متور زد ...,3,175000.0,19500000,پاترول 2در,1372
4,سمند::Samand,light,Thursday 07AM,سمند ال ایکس بدون رنگ تمامی سرویس ها انجام شده...,4,80000.0,23900000,سمند دوگانه کارخانه,1391
...,...,...,...,...,...,...,...,...,...
130438,NaN,heavy,Wednesday 03PM,بادسان فوق بیرنگ سفید رنگ بیمه تابرج 3.96 تخفی...,2,NaN,48000000,بادسان 8b14 مدل83,NaN
130439,NaN,heavy,Wednesday 06PM,باسلام .تراکتورازصفری دست خودم بوده.سالم.ودارا...,4,NaN,-1,تراکتورگلدونی ۲۳۸ایتالیا,NaN
130440,سمند::Samand,light,Wednesday 06PM,ماشینم تصادفیه با خریدار توافق میکنم سر قیمتش,3,20000.0,-1,خودرو سمند تصادفی مدل92,1392
130441,پراید صندوق‌دار::Pride,light,Thursday 03PM,سند۲ برگ،بیمه تا برج$NUMتخفیفات بیمه 15 سال،مع...,4,123000.0,6900000,پراید 79 بسیار سالم,1379
